In [4]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: IMDB Dataset.csv        


In [5]:
import pandas as pd
import numpy as np


In [6]:
df=pd.read_csv('/content/IMDB Dataset.csv')

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [9]:
df.isnull().sum()


,0
review,0
sentiment,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()


In [12]:
df['sentiment']=encoder.fit_transform(df['sentiment'])

In [13]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


positive->1
negative->0

In [14]:
df.duplicated().sum()

np.int64(418)

In [15]:
df=df.drop_duplicates(keep='first')

In [16]:
df.duplicated().sum()

np.int64(0)

In [17]:
df.shape

(49582, 2)

In [18]:
def transform_text(text):
    import re
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


In [19]:
print(transform_text('One of the other reviewers has mentioned12@@'))

one of the other reviewers has mentioned12


In [20]:
df['transformed_text']=df['review'].apply(transform_text)

In [21]:
df.head()

,review,sentiment,transformed_text
0,One of the other reviewers has mentioned that ...,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,1,a wonderful little production br br the filmin...
2,I thought this was a wonderful way to spend ti...,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,0,basically theres a family where a little boy j...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love in the time of money is a ...


In [22]:
X=df.drop(columns=['review'])

In [23]:
X.head()

,sentiment,transformed_text
0,1,one of the other reviewers has mentioned that ...
1,1,a wonderful little production br br the filmin...
2,1,i thought this was a wonderful way to spend ti...
3,0,basically theres a family where a little boy j...
4,1,petter matteis love in the time of money is a ...


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
train_data,test_data=train_test_split(X,test_size=0.2,random_state=2)

In [26]:
train_data.shape

(39665, 2)

In [27]:
test_data.shape

(9917, 2)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [30]:
from tensorflow.keras.optimizers import Adam

In [31]:
tokenize=Tokenizer(num_words=5000)
tokenize.fit_on_texts(train_data['transformed_text'])
X_train=pad_sequences(tokenize.texts_to_sequences(train_data['transformed_text']),maxlen=200)
X_test=pad_sequences(tokenize.texts_to_sequences(test_data['transformed_text']),maxlen=200)

In [32]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']

In [33]:
model=Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [35]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [36]:
model.fit(X_train,y_train,epochs=7,batch_size=64,validation_split=0.2)

Epoch 1/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 390s 768ms/step - accuracy: 0.7185 - loss: 0.5366 - val_accuracy: 0.8555 - val_loss: 0.3558
Epoch 2/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 435s 764ms/step - accuracy: 0.8379 - loss: 0.3813 - val_accuracy: 0.7194 - val_loss: 0.5399
Epoch 3/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 381s 761ms/step - accuracy: 0.8276 - loss: 0.3946 - val_accuracy: 0.8654 - val_loss: 0.3313
Epoch 4/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 383s 763ms/step - accuracy: 0.8854 - loss: 0.2845 - val_accuracy: 0.8211 - val_loss: 0.3936
Epoch 5/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 382s 763ms/step - accuracy: 0.8856 - loss: 0.2803 - val_accuracy: 0.8705 - val_loss: 0.3257
Epoch 6/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 385s 776ms/step - accuracy: 0.9070 - loss: 0.2331 - val_accuracy: 0.8765 - val_loss: 0.3288
Epoch 7/7
496/496 ━━━━━━━━━━━━━━━━━━━━ 438s 767ms/step - accuracy: 0.9249 - loss: 0.1986 - val_accuracy: 0.8799 - val_loss: 0.3330


In [37]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


310/310 ━━━━━━━━━━━━━━━━━━━━ 40s 127ms/step - accuracy: 0.8799 - loss: 0.3144
Loss: 0.324802964925766
Accuracy: 0.8802056908607483


In [38]:
model.save("sentiment_model.h5")


In [39]:
model.save("sentiment_model.keras")


In [40]:

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenize, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
import pickle

# Save tokenizer (make sure 'tokenizer' is already fitted)
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenize, f)


In [43]:
with open("config.txt", "w") as f:
    f.write("maxlen=200")
